# Challenge 1

### Imports

In [6]:
import requests
import os
import zipfile

### Log in to hackthissite.com

### List of scrambled words

In [7]:
import nest_asyncio
from requests_html import AsyncHTMLSession
import asyncio

nest_asyncio.apply()  # Allows running nested async functions in Jupyter

async def fetch_scrambled_words():
    url = "https://www.hackthissite.org/missions/prog/1/"
    session = AsyncHTMLSession()
    response = await session.get(url)
    await response.html.arender()  # Render JavaScript

    scrambled_words = [li.text.strip() for li in response.html.find("td li")]

    if scrambled_words:
        print("Scrambled words found:")
        for word in scrambled_words:
            print(word)
    else:
        print("No scrambled words found.")

    await session.close()

# Run the async function in the existing event loop
asyncio.create_task(fetch_scrambled_words())


<Task pending name='Task-2' coro=<fetch_scrambled_words() running at C:\Users\fjun2024\AppData\Local\Temp\ipykernel_5376\2958366337.py:7>>

### Download wordlist

In [3]:
url = "https://www.hackthissite.org/missions/prog/1/wordlist.zip"
file_name = "wordlist.zip"

# Check if the file already exists
if not os.path.exists(file_name):
    # Send GET request
    response = requests.get(url, stream=True)
    
    # Check if the request was successful
    if response.status_code == 200:
        with open(file_name, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                file.write(chunk)
        print(f"File downloaded successfully as {file_name}")
    else:
        print(f"Failed to download file. HTTP Status Code: {response.status_code}")
else:
    print(f"File '{file_name}' already exists. Skipping download.")


File 'wordlist.zip' already exists. Skipping download.


### Unzip file

In [4]:
# Extract the ZIP file if not already extracted
extract_folder = "wordlist"
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        print(f"File extracted to {extract_folder}")
else:
    print(f"Folder '{extract_folder}' already exists. Skipping extraction.")



Folder 'wordlist' already exists. Skipping extraction.


### Read the words

In [5]:
# Open and read the extracted file into a list, line by line
word_list = []
wordlist_path = os.path.join(extract_folder, "wordlist.txt")
if os.path.exists(wordlist_path):
    with open(wordlist_path, "r", encoding="utf-8") as file:
        for line in file:
            word_list.append(line.strip())
            # print(line.strip())  # Process each line as needed
else:
    print(f"Extracted wordlist file '{wordlist_path}' not found.")
print(word_list)

['html:)', '121212', '131313', '123123', '654321', '8675309', '666666', '696969', '888888', '1234567', '21122112', '12345678', 'asdfjkl;', 'hal9000', 'bond007', 'ncc1701d', 'ncc1701e', 'ncc1701', 'thx1138', 'a12345', 'abcd1234', '1234qwer', '1a2b3c', '1q2w3e', 'test123', '1p2o3i', 'puppy123', 'kitten12', 'qwerty12', 'john316', 'apollo13', 'ne1469', 'amanda1', 'mazda1', 'angela1', 'alpha1', 'sarah1', 'nirvana1', 'bubba1', 'scuba1', 'rambo1', 'charlie1', 'david1', 'digital1', 'dragon1', 'honda1', 'shadow1', 'eagle1', 'freak1', 'james1', '1sanjose', 'apple1', 'master1', 'happy1', 'martin1', 'jason1', 'larry1', 'number1', 'robert1', 'soccer1', 'direct1', 'chester1', 'welcome1', 'french1', 'hockey1', 'chevy1', 'scooter1', 'chris1', 'lucky1', 'teddy1', 'phoenix1', 'hello1', 'julie1', 'kevin1', 'pookie1', 'viper1', 'jenny1', 'jesus1', 'kelly1', 'money1', 'mouse1', 'sting1', 'justin1', 'molly1', 'sunny1', 'front242', 'jordan23', '2welcome', 'h2opolo', 'bird33', '4runner', 'babylon5', 'star69',